# Simulating a projectile with and without air resistance 

Safwan Ahmed 12/12/2022

As a projectile is launched it experinces mutiple forces including gravity and air resistance. Normally we ignore air resistance as it's complicated to account for. However in reality, we need to take it into account. The formula for air resistance is given by 

$$
\text{Air resistance} = - k v \mathbf{v}
$$

Where $v$ is the magnitude of the velocity vector $\mathbf{v}$, and $k$ is a constant which is simplified from the full equation that contains particles mass, shape and fluid it's going through


## Introduction
 
First we will we will be using Euler's formula to simulate a particle without Air resistance. 

First Import libraries as always 

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from IPython.display import HTML

In [2]:
def animate_with_trail(i, bodies, trails, x_arr, y_arr):
    """
    Update display for projectile motion, with trail.
    
    Arguments:
        bodies: Line2D object containing coordinates of bodies to move
        trails: Line2D object containing coordinates of trails
        i:      frame number (from 0 at time = 0)
        x_arr:  array of x coordinate at each time step
        y_arr:  array of y coordinate at each time step

    Result: updates coordinates in Line2D objects provided as arguments
    "bodies" and "trails".
    """
    x, y = x_arr[i], y_arr[i]
    bodies.set_data([x],[y])                 # Body gets coordinates of current position
    trails.set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to the current one

In [17]:

v0    = 10       # Launch speed [m/s]
theta = 45       # Launch angle [degrees]
dt    = 0.01     # Time step [s]
t_max = 2.       # End time of calculation [s]



In [14]:
def move_projectile(position, velocity, dt):
    """
    Update the position and velocity of a projectile after an additional time step dt.
    """
    g = -9.81              # acceleration due to gravity [m/s^2] (negative = downwards)
    accel = np.array([0,g])
    pos_new = position + velocity * dt    # r = r + v dt
    vel_new = velocity + accel    * dt    # v = v + a dt
    return pos_new, vel_new

In [15]:
def trajectory_euler(v0, angle, dt, t_max):
    """
    Calculate trajectory of projectile launched from the origin at given angle and speed,
    stopping when the projectile reaches the ground, or when a time limit is reached if
    this is earlier.
    
    Arguments:
        v0:    launch speed [m/s]
        angle: launch angle from horizontal [degrees]
        dt:    time step [s]
        t_max: maximum end time of calculation [s]
    
    Returns: NumPy arrays of coordinates x_array, y_array [m]
    """
    position = np.array([0., 0.])
    theta = np.deg2rad(angle)                            # convert angle to radians
    velocity = v0 * np.array([np.cos(theta), np.sin(theta)])
    t = 0.
    x, y = position
    x_arr, y_arr = [x], [y]
    while y>=0:
        t += dt
        position, velocity = move_projectile(position, velocity, dt)
        x, y = position
        x_arr.append(x)
        y_arr.append(y)
    return x_arr, y_arr

In [16]:
# Calculate trajectory using Euler's method, without air resistance
x_eul, y_eul = trajectory_euler(10,45,dt,10)

# Create and configure figure and axes
plt.ioff()
fig, axes = plt.subplots()
axes.set_xlim(-1,11)
axes.set_ylim(0,5)
axes.set_aspect('equal')

# Create objects to represent projectile position and trail
projectile, = axes.plot([],[],'o')
trail,      = axes.plot([],[],'-')

# Create animation
ani4 = animation.FuncAnimation(fig,animate_with_trail, frames=len(x_eul), interval=dt*1000,
                               fargs=(projectile, trail, x_eul, y_eul))
HTML(ani4.to_jshtml())

As we increase dt, the number of frames decreases and the motion looks less smoooth as the frame lasts for longer. For dt, 0.01 seconds is a good value as it allows the motion to look smooth whilst not putting a massive computational load on our device. 

Now we will model with Air resistance. Air resistance acts in both the horizontal and vertical direction. We will use Euler's method to model our particle. Even though the equation for air resistance is $- k v \mathbf{v}$, $\mathbf{v}$ here is just a vector. Its much easier if we just make them seperate in their respective directions while coding it.

In [7]:
k = 0.5 #Define k which is a constant that we can vary

def move_proj_air_res(position, velocity, dt, k):
    """
    Update the position and velocity of a projectile after an additional time step dt. Considers air resistance.
    """
    g = -9.81              # acceleration due to gravity [m/s^2] (negative = downwards)
    accel = np.array([0,g])
    pos_new = position + velocity * dt + 0.5 * -k * velocity * dt ** 2  # r = r + v dt + 0.5 * -kv dt^2
    vel_new = velocity + accel    * dt + -k * velocity * dt   # v = v + a dt - kv
    return pos_new, vel_new
def trajectory_euler_air_res(v0, angle, dt, t_max):
    """
    Calculate trajectory of projectile launched from the origin at given angle and speed,
    stopping when the projectile reaches the ground, or when a time limit is reached if
    this is earlier. Considers air resistance.
    
    Arguments:
        v0:    launch speed [m/s]
        angle: launch angle from horizontal [degrees]
        dt:    time step [s]
        t_max: maximum end time of calculation [s]
    
    Returns: NumPy arrays of coordinates x_array, y_array [m]
    """
    position = np.array([0., 0.])
    theta = np.deg2rad(angle)                            # convert angle to radians
    velocity = v0 * np.array([np.cos(theta), np.sin(theta)])
    t = 0.
    x, y = position
    x_arr, y_arr = [x], [y]
    while y>=0:
        t += dt
        position, velocity = move_proj_air_res(position, velocity, dt, k)
        x, y = position
        x_arr.append(x)
        y_arr.append(y)
    return x_arr, y_arr
# Calculate trajectory using Euler's method, without air resistance
x_eul, y_eul = trajectory_euler_air_res(10,45,dt,10)

# Create and configure figure and axes
plt.ioff()
fig, axes = plt.subplots()
axes.set_xlim(-1,11)
axes.set_ylim(0,5)
axes.set_aspect('equal')

# Create objects to represent projectile position and trail
projectile, = axes.plot([],[],'o')
trail,      = axes.plot([],[],'-')

# Create animation
ani4 = animation.FuncAnimation(fig,animate_with_trail, frames=len(x_eul), interval=dt*1000,
                               fargs=(projectile, trail, x_eul, y_eul))
HTML(ani4.to_jshtml())

When K is zero, the motion is identical to the simulation before, which is what we expect as it has no air resistance. As we increase k, the particle doesn't reach as high and also falls off quicker. The trajectory also is no longer symmetrical as the particle is slower after reaching its highest point, hence the horizontal distance it covers is also lower. 

Using Eulers method is not neccessary as its possible to find an analytical solution for the motion of the particle. But as long as dt is small enough, it provides a very good approximation to the actual motion of the particle.

In [8]:
###########
#REFERENCES
###########

#Code accessed from 'Unit 9 notes' by Ben Waugh, Becky Chislett